In [10]:
# Imports
import numpy as np
import pandas as pd
import scipy.stats as stats
from numpy.linalg import inv
from scipy.linalg import solve
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from statsmodels.graphics.tsaplots import plot_acf
import seaborn as sns
from tqdm import tqdm
from numpy.linalg import eigvals

# Load data
file_path = "Yields_Bloomberg.xlsx"
real_data = pd.read_excel(file_path, index_col=0)

# Map columns to maturities
maturity_map = {
    "TREASURY0,25": 3, "TREASURY0,5": 6, "TREASURY0,75": 9, "TREASURY1": 12,
    "TREASURY1,25": 15, "TREASURY1,5": 18, "TREASURY1,75": 21, "TREASURY2": 24,
    "TREASURY2,5": 30, "TREASURY3": 36, "TREASURY4": 48, "TREASURY5": 60,
    "TREASURY6": 72, "TREASURY7": 84, "TREASURY8": 96, "TREASURY9": 108,
    "TREASURY10": 120, "TREASURY15": 180, "TREASURY20": 240, "TREASURY30": 360
}
real_data = real_data[[col for col in maturity_map.keys()]]
taus = np.array([maturity_map[col] for col in real_data.columns])
yields = real_data.to_numpy()[:200]
T, N = yields.shape

In [11]:
# NS loadings
def nelson_siegel_loadings(taus, lambd):
    L = np.zeros((len(taus), 3))
    for i, tau in enumerate(taus):
        L[i, 0] = 1
        L[i, 1] = (1 - np.exp(-lambd * tau)) / (lambd * tau)
        L[i, 2] = L[i, 1] - np.exp(-lambd * tau)
    return L

# FFBS
def ffbs_stable(yields, L, A, mu, Q, H):
    T, N = yields.shape
    m = 3
    betas = np.zeros((T, m))
    P = np.zeros((T, m, m))
    beta_pred = mu
    P_pred = Q
    I = np.eye(m)
    for t in range(T):
        y_t = yields[t]
        S = L @ P_pred @ L.T + H + 1e-6 * np.eye(N)
        K = solve(S, L @ P_pred.T, assume_a='pos').T
        beta_filt = beta_pred + K @ (y_t - L @ beta_pred)
        P_filt = (I - K @ L) @ P_pred
        betas[t] = beta_filt
        P[t] = P_filt
        if t < T - 1:
            beta_pred = mu + A @ (beta_filt - mu)
            P_pred = A @ P_filt @ A.T + Q
    beta_draws = np.zeros((T, m))
    beta_draws[-1] = np.random.multivariate_normal(betas[-1], P[-1] + 1e-6 * I)
    for t in reversed(range(T-1)):
        P_f = P[t]
        J = P_f @ A.T @ inv(A @ P_f @ A.T + Q + 1e-6 * I)
        mean = betas[t] + J @ (beta_draws[t+1] - mu - A @ (betas[t] - mu))
        cov = P_f - J @ A @ P_f
        beta_draws[t] = np.random.multivariate_normal(mean, cov + 1e-6 * I)
    return beta_draws

# Bayesian sampling of mu, A, Q
def sample_mu_A_Q_bayesian(betas):
    mu_prior_mean = np.array([6.0, -2.5, -1.3])
    mu_prior_cov = np.diag([2.5**2, 1.5**2, 2.7**2])
    nu_Q = 5
    S_Q = np.eye(3)
    A0 = np.eye(3)
    Sigma_A = 0.01 * np.eye(3)
    T = betas.shape[0]
    X = betas[:-1]
    Y = betas[1:]
    Xc = X - X.mean(axis=0)
    Yc = Y - Y.mean(axis=0)
    Sxy = Xc.T @ Yc
    Sxx = Xc.T @ Xc
    A_hat = Sxy @ inv(Sxx + 1e-6 * np.eye(3))
    residuals = Yc - Xc @ A_hat
    S = residuals.T @ residuals
    Q_post = stats.invwishart.rvs(df=nu_Q + T - 1, scale=S + S_Q)
    precision_prior = inv(mu_prior_cov)
    precision_lik = T * inv(Q_post)
    cov_post = inv(precision_prior + precision_lik)
    mean_lik = inv(Q_post) @ (Y.mean(axis=0) - A_hat @ X.mean(axis=0))
    mean_post = cov_post @ (precision_prior @ mu_prior_mean + precision_lik @ mean_lik)
    mu_post = np.random.multivariate_normal(mean_post, cov_post)
    Xc = X - mu_post
    Yc = Y - mu_post
    V_n = inv(inv(Sigma_A) + Xc.T @ Xc)
    M_n = V_n @ (inv(Sigma_A) @ A0 + Xc.T @ Yc)
    A_post = np.zeros((3, 3))
    for i in range(3):
        A_post[i, :] = np.random.multivariate_normal(M_n[i, :], Q_post * V_n[i, i])
    return mu_post, A_post, Q_post

# Sample H
def sample_H(yields, betas, lambd):
    a_H, b_H = 2.0, 0.5
    T, N = yields.shape
    L = nelson_siegel_loadings(taus, lambd)
    H_diag = np.zeros(N)
    for i in range(N):
        eps = yields[:, i] - betas @ L[i]
        shape = a_H + T / 2
        scale = b_H + 0.5 * np.sum(eps**2)
        H_diag[i] = stats.invgamma.rvs(a=shape, scale=scale)
    return np.diag(H_diag)

# Sample lambda
def sample_lambda(yields, betas, lambda_curr, H, proposal_std=0.005):
    alpha_lambda, beta_lambda = 5.5, 15
    lambda_prop = abs(lambda_curr + np.random.normal(0, proposal_std))
    L_curr = nelson_siegel_loadings(taus, lambda_curr)
    L_prop = nelson_siegel_loadings(taus, lambda_prop)
    loglik_curr = -0.5 * np.sum((yields - betas @ L_curr.T)**2 / np.diag(H))
    loglik_prop = -0.5 * np.sum((yields - betas @ L_prop.T)**2 / np.diag(H))
    logprior_curr = stats.gamma.logpdf(lambda_curr, alpha_lambda, scale=1 / beta_lambda)
    logprior_prop = stats.gamma.logpdf(lambda_prop, alpha_lambda, scale=1 / beta_lambda)
    log_accept_ratio = (loglik_prop + logprior_prop) - (loglik_curr + logprior_curr)
    return lambda_prop if np.log(np.random.rand()) < log_accept_ratio else lambda_curr

In [13]:
# Initialize MCMC
num_iters = 1500
burn_in = 500
lambda_samples, A_samples, mu_samples, Q_samples, H_samples = [], [], [], [], []
lambda_curr = 0.06
mu = np.zeros(3)
A = np.eye(3) * 0.95
Q = 0.01 * np.eye(3)
H = 0.01 * np.eye(N)
betas = np.zeros((T, 3))

# Run MCMC
for _ in tqdm(range(num_iters)):
    L = nelson_siegel_loadings(taus, lambda_curr)
    betas = ffbs_stable(yields, L, A, mu, Q, H)
    mu, A, Q = sample_mu_A_Q_bayesian(betas)
    H = sample_H(yields, betas, lambda_curr)
    lambda_curr = sample_lambda(yields, betas, lambda_curr, H)
    lambda_samples.append(lambda_curr)
    A_samples.append(A.copy())
    mu_samples.append(mu.copy())
    Q_samples.append(Q.copy())
    H_samples.append(H.copy())

# Convert to arrays
lambda_samples = np.array(lambda_samples)[burn_in:]
A_samples = np.array(A_samples)[burn_in:]
mu_samples = np.array(mu_samples)[burn_in:]
Q_samples = np.array(Q_samples)[burn_in:]
H_samples = np.array(H_samples)[burn_in:]

  3%|▎         | 45/1500 [00:01<00:54, 26.49it/s]C:\Users\ricar\AppData\Local\Temp\ipykernel_22104\1197894924.py:31: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  beta_draws[-1] = np.random.multivariate_normal(betas[-1], P[-1] + 1e-6 * I)
C:\Users\ricar\AppData\Local\Temp\ipykernel_22104\1197894924.py:37: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  beta_draws[t] = np.random.multivariate_normal(mean, cov + 1e-6 * I)
100%|██████████| 1500/1500 [00:53<00:00, 27.88it/s]


In [23]:
#import pyreadr
import json

results = {
    "lambda": lambda_samples.tolist(),
    "A": A_samples.tolist(),
    "mu": mu_samples.tolist(),
    "Q": Q_samples.tolist(),
    "H": H_samples.tolist()
}

with open("output.json", "w") as json_file:
    json.dump(results, json_file)